In [2]:
!pip install transformers
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.0
    Uninstalling tensorflow-2.12.0:
     

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="noelmathewisaac/inspirational-quotes-distilgpt2")

2023-10-26 00:21:50.428499: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-26 00:21:50.428566: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-26 00:21:50.428615: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
Arg specs do not match: original=FullArgSpec(args=['input', 'dtype', 'name', 'layout'], varargs=None, varkw=None, defaults=(None, None, None), kwonlyargs=[], kwonlydefaults=None, annotations={}), new=FullArgSpec(args=['input', 'dtype', 'name'], varargs=None, varkw=None, defaults=(None, None), kwonlyargs=[], kwonlydefaults=None, annotations={}), fn=<function ones_like_v2 at 0x79a811d20430>

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("noelmathewisaac/inspirational-quotes-distilgpt2")
model = AutoModelForCausalLM.from_pretrained("noelmathewisaac/inspirational-quotes-distilgpt2")

file_path = '/content/drive/My Drive/prepro/generated_quotes.csv'

# Open the file in write mode to clear its content
with open(file_path, 'w') as file:
    file.truncate(0)


In [ ]:
import random
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

# Check if a GPU is available and use it
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("noelmathewisaac/inspirational-quotes-distilgpt2")
model = AutoModelForCausalLM.from_pretrained("noelmathewisaac/inspirational-quotes-distilgpt2").to(device)

# Define the path to your dataset in Kaggle
dataset_path = '/kaggle/input/sammyyy/preprocessed_quotes_no_quote_author_category (1).csv'

# Load the preprocessed dataset
preprocessed_data = pd.read_csv(dataset_path)

# Define the path to save the generated file in Kaggle's working directory
generated_file_path = '/kaggle/working/generated_quotes (1).csv'

# Function to generate quotes for each row without overwriting existing quotes
def generate_quotes_for_each_row(dataframe, max_rows, generated_file_path, max_length=100, save_every=100):
    generated_quotes = []

    if os.path.exists(generated_file_path):
        # Load existing generated quotes and determine the starting index in the dataset
        existing_quotes_data = pd.read_csv(generated_file_path, delimiter='\t')
        num_existing_quotes = len(existing_quotes_data)
    else:
        num_existing_quotes = 0

    for index in range(num_existing_quotes, max_rows):
        entry = dataframe.iloc[index]["first_3_words"]
        seed_text = entry
        input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)
        attention_mask = torch.ones(input_ids.shape, device=device)
        output = model.generate(input_ids, max_length=max_length, no_repeat_ngram_size=20, top_k=50, pad_token_id=model.config.eos_token_id, attention_mask=attention_mask)
        quote = tokenizer.decode(output[0], skip_special_tokens=True)

        generated_quotes.append({"Generated_Quote": quote, "Source_Row": f"Row {index + 1}"})

        if len(generated_quotes) % save_every == 0:
            # Append newly generated quotes to the existing file every 100 quotes
            generated_quotes_data = pd.DataFrame(generated_quotes)
            generated_quotes_data.to_csv(generated_file_path, mode='a', header=False, index=False, sep='\t')
            generated_quotes = []  # Clear the list to avoid saving the same quotes multiple times
            print(f"Yay {index + 1} saved")

    # Append any remaining newly generated quotes
    if generated_quotes:
        generated_quotes_data = pd.DataFrame(generated_quotes)
        generated_quotes_data.to_csv(generated_file_path, mode='a', header=False, index=False, sep='\t')
        print(f"Yay {max_rows} saved")

# Generate up to 1 quote per row for the maximum number of rows
max_rows_to_generate = len(preprocessed_data)
generate_quotes_for_each_row(preprocessed_data, max_rows_to_generate, generated_file_path)


In [1]:
import shutil

# Source file path
source_file = '/kaggle/input/sammyyy/generated_quotes (1).csv'

# Destination file path (working directory)
destination_file = '/kaggle/working/generated_quotes (1).csv'

# Copy the file to the working directory
shutil.copy(source_file, destination_file)


'/kaggle/working/generated_quotes (1).csv'